<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week11.do.txt --no_mako -->
<!-- dom:TITLE: April 1-4, 2024: Quantum Computing, Quantum Machine Learning and Quantum Information Theories -->

# April 1-4, 2024: Quantum Computing, Quantum Machine Learning and Quantum Information Theories
**Morten Hjorth-Jensen**, Department of Physics, University of Oslo and Department of Physics and Astronomy and Facility for Rare Isotope Beams, Michigan State University

Date: **April 1, 2024**

## Plans for the week of April 1-4, 2024

1. Quantum Fourier transforms (QFT), basic mathematical expressions

2. Setting up circuits for QFT

3. Reading recommendation Hundt, Quantum Computing for Programmers, sections 6.1-6.4 on QFT.
<!-- o [Whiteboard notes](https://github.com/CompPhysics/QuantumComputingMachineLearning/blob/gh-pages/doc/HandWrittenNotes/2024/NotesMarch20.pdf) -->

## Possible paths for project 2

* Implement QFTs  and study the phase estimation algorithm and eventually Shor's algorithm for factorization of numbers.

* Study other algorithms

a. Deutsch-Jozsa algorithm: Determine if a function is constant or balance using the fewest number of queries.

b. SImon's algorithm: Determine if a function (Oracle) is one-to-one or two-to-one

c. Grover's algorithm: Search unstructured list of data fast

d. Shor's algorithm : Factorize integers efficiently

* Study the solution of quantum mechanical eigenvalue problems with systems from atomic/molecular physics and quantum chemistry

* Quantum machine learning projects

* Other paths

For project 2, in order to be time efficient, you can use software like Qiskit, Pennylane, qBraid and/or other

## Brief reminder on Fourier transforms

## The discrete Fourier transform (DFT)
The discrete Fourier transform takes as input a complex vector

$$
\begin{align*}
\boldsymbol{x} =\vert\boldsymbol{x}\rangle= 
\begin{bmatrix}
x_0 \\ 
x_1 \\ 
x_2 \\
\vdots \\ 
x_{n-2} \\
x_{n-1}
\end{bmatrix}.
\end{align*}
$$

Each entry of the output vector $\vert \boldsymbol{y}\rangle$ is given by

$$
\begin{align*}
y_k = \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} x_je^{2 \pi ijk/N}.
\end{align*}
$$

## Output vector

The output is a complex vector

$$
\begin{align*}
\vert \boldsymbol{y}\rangle = 
\frac{1}{\sqrt{n}}
\begin{bmatrix}
\sum_{j=0}^{n-1} x_je^{2\pi ij0/n} \\
\sum_{j=0}^{n-1} x_je^{2\pi ij1/n} \\
\sum_{j=0}^{n-1} x_je^{2\pi ij2/n} \\
\vdots \\
\sum_{j=0}^{n-1} x_je^{2\pi ij(n-2)/n} \\
\sum_{j=0}^{n-1} x_je^{2\pi ij(n-1)/n}
\end{bmatrix}
\end{align*}
$$

## Discrete Fourier Transformations

Consider two sets of complex numbers $x_k$ and $y_k$ with
$k=0,1,\dots,n-1$ entries. The discrete Fourier transform is defined
as

$$
y_k = \frac{1}{\sqrt{n-1}} \sum_{j=0}^{n-1} \exp{(\frac{2\pi\imath jk}{n})} x_j.
$$

As an example, assume $x_0=1$ and $x_1=1$. We can then use the above expression to find $y_0$ and $y_1$.

With the above formula we get then

$$
y_0 = \frac{1}{\sqrt{2}} \left( \exp{(\frac{2\pi\imath 0\times 1}{2})} \times 1+\exp{(\frac{2\pi\imath 0\times 1}{2})}\times 2\right)=\frac{1}{\sqrt{2}}(1+2)=\frac{3}{\sqrt{2}},
$$

and

$$
y_1 = \frac{1}{\sqrt{2}} \left( \exp{(\frac{2\pi\imath 0\times 1}{2})} \times 1+\exp{(\frac{2\pi\imath 1\times 1}{2})}\times 2\right)=\frac{1}{\sqrt{2}}(1+2\exp{(\pi\imath)})=-\frac{1}{\sqrt{2}},
$$

## More details on Discrete Fourier transforms

Suppose that we have a vector $f$ of $N$ complex numbers, $f_{k}, k
\in\{0,1, \ldots, N-1\}$. Then the discrete Fourier transform (DFT) is
a map from these $N$ complex numbers to $N$ complex numbers, the
Fourier transformed coefficients $\tilde{f}_{j}$, given by

$$
\tilde{f}_{j}=\frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} \omega^{-j k} f_{k}
$$

where $\omega=\exp \left(\frac{2 \pi i}{N}\right)$.

## Invert DFT
The inverse DFT is given by

$$
f_{j}=\frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} \omega^{j k} \tilde{f}_{k}
$$

To see this consider how the basis vectors transform. If $f_{k}^{l}=\delta_{k, l}$, then

$$
\tilde{f}_{j}^{l}=\frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} \omega^{-j k} \delta_{k, l}=\frac{1}{\sqrt{N}} \omega^{-j l}
$$

## Orthonormality
These DFT vectors are orthonormal:

$$
\sum_{j=0}^{N-1} \tilde{f}^{l}{ }_{j}^{*} \tilde{f}_{j}^{m}=\frac{1}{N} \sum_{j=0}^{N-1} \omega^{j l} \omega^{-j m}=\frac{1}{N} \sum_{j=0}^{N-1} \omega^{j(l-m)}
$$

This last sum can be evaluated as a geometric series, but beware of the $(l-m)=0$ term, and yields

$$
\sum_{j=0}^{N-1} \tilde{f}^{l}{ }_{j}^{*} \tilde{f}_{j}^{m}=\delta_{l, m}
$$

From this we can check that the inverse DFT does indeed perform the inverse transform:

$$
f_{j}=\frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} \omega^{j k} \tilde{f}_{k}=\frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} \omega^{j k} \frac{1}{\sqrt{N}} \sum_{l=0}^{N-1} \omega^{-l k} f_{l}=\frac{1}{N} \sum_{k, l=0}^{N-1} \omega^{(j-l) k} f_{l}=\sum_{l=0}^{N-1} \delta_{j, l} f_{l}=f_{j}
$$

## Quantum Fourier transform

We now turn to the quantum Fourier transform. It is the same
transformation as described above, however we define it in terms of
the unitary operation

$$
\vert \psi'\rangle \leftarrow \hat{F}\vert \psi\rangle, \quad \hat{F}^\dagger \hat{F} = I
$$

## Orthonormal basis

In terms of an orthonormal basis $\vert 0 \rangle,\vert 1\rangle,\dots,\vert 0 \rangle$ this linear operator has the following action

$$
\vert j \rangle \rightarrow \sum_{k=0}^{N-1} \exp{i(2\pi jk/N)}\vert k
$$

or on an arbitrary state

$$
\sum_{j=0}^{N-1} x_j \vert j \rangle \rightarrow \sum_{k=0}^{N-1} y_k\vert k \rangle
$$

equivalent to the equation for discrete Fourier transform on a set of complex numbers.

## Using computational basis

Next we assume an $n$-qubit system, where we take $N=s^n$ in the computational basis

$$
\vert 0 \rangle,\dots,\vert 2^n -1\rangle.
$$

We make use of the binary representation $j = j_1 2^{n-1} + j_2
2^{n-2} + \dots + j_n 2^0$ , and take note of the notation $0.j_l
j_{l+1} \dots j_m$ representing the binary fraction $\frac{j_l}{2^1} +
\frac{j_{l+1}}{2^{2}} + \dots + \frac{j_m}{2^{m-l+1}}$. With this we
define the product representation of the quantum Fourier transform

$$
\vert j_1,\dots,j_n\rangle  \rightarrow 
\frac{
\left(\vert 0 \rangle + \exp{i(2\pi 0.j_n)}\right)
\left(\vert 0 \rangle + \exp{i(2\pi 0.j_{j-1}j_n)}\right)
\dots
\left(\vert 0 \rangle + \exp{i(2\pi 0.j_1j_2\dots j_n)}\right)
}{2^{n/2}}
$$

## Components

From the product representation we can derive a circuit for the
quantum Fourier transform. This will make use of the following two
single-qubit gates

$$
H = \frac{1}{\sqrt{2}}
    \begin{bmatrix}
        1 & 1 \\
        1 & -1
    \end{bmatrix}
$$

$$
R_k =
    \begin{bmatrix}
        1 & 0 \\
        0 & e^{2\pi i/2^{k}}
    \end{bmatrix}
$$

## Using the Hadamard gate
The Hadamard
gate on a single qubit creates an equal superposition of its basis
states, assuming it is not already in a superposition, such that

$$
H\vert 0 \rangle = \frac{1}{\sqrt{2}} \left(\vert 0 \rangle + \vert 1\rangle\right), \quad H\vert 1\rangle = \frac{1}{\sqrt{2}} \left(\vert 0 \rangle - \vert 1\rangle\right)
$$

The $R_k$ gate simply adds a phase if the qubit it acts on is in the state $\vert 1\rangle$

$$
R_k\vert 0 \rangle = \vert 0 \rangle, \quad R_k\vert 1\rangle = e^{2\pi i/2^{k}}\vert 1\rangle
$$

Since all this gates are unitary, the quantum Fourier transfrom is also unitary.

## Algorithm

Assume we have a quantum register of $n$ qubits in the state $\vert j_1 j_2 \dots j_n\rangle$.
Applying the Hadamard gate to the first qubit
produces the state

$$
H\vert j_1 j_2 \dots j_n\rangle = \frac{\left(\vert 0 \rangle + e^{2\pi i 0.j_1}\vert 1\rangle\right)}{2^{1/2}} \vert j_2 \dots j_n\rangle.
$$

## Binary fraction

Here we have made use of the binary fraction to represent the action of the Hadamard gate

$$
\exp{2\pi i 0.j_1} = -1,
$$

if $j_1 = 1$ and $+1$ if $j_1 = 0$.

## Controlled rotation gate

Furthermore we can apply the controlled-$R_k$ gate, with all the other qubits $j_k$ for $k>1$ as control qubits to produce the state

$$
\frac{\left(\vert 0 \rangle + e^{2\pi i 0.j_1j_2\dots j_n}\vert 1\rangle\right)}{2^{1/2}} \vert j_2 \dots j_n\rangle
$$

Next we do the same procedure on qubit $2$ producing the state

$$
\frac{\left(\vert 0 \rangle + e^{2\pi i 0.j_1j_2\dots j_n}\vert 1\rangle\right)\left(\vert 0 \rangle + e^{2\pi i 0.j_2\dots j_n}\vert 1\rangle\right)}{2^{2/2}} \vert j_2 \dots j_n\rangle
$$

## Applying to all qubits

Doing this for all $n$ qubits yields state

$$
\frac{\left(\vert 0 \rangle + e^{2\pi i 0.j_1j_2\dots j_n}\vert 1\rangle\right)\left(\vert 0 \rangle + e^{2\pi i 0.j_2\dots j_n}\vert 1\rangle\right)\dots \left(\vert 0 \rangle + e^{2\pi i 0.j_n}\vert 1\rangle\right)}{2^{n/2}} \vert j_2 \dots j_n\rangle
$$

At the end we use swap gates to reverse the order of the qubits

$$
\frac{\left(\vert 0 \rangle + e^{2\pi i 0.j_n}\vert 1\rangle\right)\left(\vert 0 \rangle + e^{2\pi i 0.j_{n-1}j_n}\vert 1\rangle\right)\dots\left(\vert 0 \rangle + e^{2\pi i 0.j_1j_2\dots j_n}\vert 1\rangle\right) }{2^{n/2}} \vert j_2 \dots j_n\rangle
$$

This is just the product representation from earlier, obviously our desired output.

## Quantum Fourier transform

Now lets turn to the Quantum Fourier transform (QFT). We've already
seen the QFT for $N=2$. It is the Hadamard transform:

$$
H=\frac{1}{\sqrt{2}}\left[\begin{array}{cc}
1 & 1  \\
1 & -1
\end{array}\right]
$$

## QFT for $N=2$

Why is this the QFT for $N=2$ ? Well suppose have the single qubit
state $a_{0}|0\rangle+a_{1}|1\rangle$. If we apply the Hadamard
operation to this state we obtain the new state

$$
\frac{1}{\sqrt{2}}\left(a_{0}+a_{1}\right)|0\rangle+\frac{1}{\sqrt{2}}\left(a_{0}-a_{1}\right)|1\rangle=\tilde{a}_{0}|0\rangle+\tilde{a}_{1}|1\rangle .
$$

In other words the Hadamard gate performs the DFT for $N=2$ on the
amplitudes of the state! Notice that this is very different that
computing the DFT for $N=2$ : remember the amplitudes are not numbers
which are accessible to us mere mortals, they just represent our
description of the quantum system.

## Full QFT

So what is the full quantum Fourier transform? It is the transform
which takes the amplitudes of a $N$ dimensional state and computes the
Fourier transform on these amplitudes (which are then the new
amplitudes in the computational basis.) In other words, the QFT enacts
the transform

$$
\sum_{x=0}^{N-1} a_{x}|x\rangle \rightarrow \sum_{x=0}^{N-1} \tilde{a}_{x}|x\rangle=\sum_{x=0}^{N-1} \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} \omega_{N}^{-x y} a_{y}|x\rangle
$$

## Explicit transform
It is easy to see that this implies that the QFT performs the following transform on basis states:

$$
|x\rangle \rightarrow \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} \omega_{N}^{-x y}|y\rangle
$$

Thus the QFT is given by the matrix

$$
U_{Q F T}=\frac{1}{\sqrt{N}} \sum_{x=0}^{N-1} \sum_{y=0}^{N-1} \omega_{N}^{-y x}|y\rangle\langle x|
$$

## Unitarity
The last  matrix is unitary. Let's check this:

$$
\begin{align*}
U_{Q F T} U_{Q F T}^{\dagger} & =\frac{1}{N} \sum_{x=0}^{N-1} \sum_{y=0}^{N-1} \omega_{N}^{y x}|x\rangle\left\langle y\left|\sum_{x^{\prime}=0}^{N-1} \sum_{y^{\prime}=0}^{N-1} \omega_{N}^{-y^{\prime} x^{\prime}}\right| y^{\prime}\right\rangle\left\langle x^{\prime}\right| \\
& =\frac{1}{N} \sum_{x, y, x^{\prime}, y^{\prime}=0}^{N-1} \omega_{N}^{y x-y^{\prime} x^{\prime}} \delta_{y, y^{\prime}}|x\rangle\left\langle x^{\prime}\left|=\frac{1}{N} \sum_{x, y, x^{\prime}=0}^{N-1} \omega_{N}^{y\left(x-x^{\prime}\right)}\right| x\right\rangle\left\langle x^{\prime}\right| \\
& =\sum_{x, x^{\prime}=0}^{N-1} \delta_{x, x^{\prime}}|x\rangle\left\langle x^{\prime}\right|=I 
\end{align*}
$$

## Importance of QFT

The QFT is a very important transform in quantum computing. It can be
used for all sorts of cool tasks, including, as we shall see in Shor's
algorithm. But before we can use it for quantum computing tasks, we
should try to see if we can efficiently implement the QFT with a
quantum circuit. Indeed we can and the reason we can is intimately
related to the fast Fourier transform.

## Circuit QFT

Let's derive a circuit for the QFT when $N=2^{n}$. The QFT performs the transform

$$
|x\rangle \rightarrow \frac{1}{\sqrt{2^{n}}} \sum_{y=0}^{2^{n}-1} \omega_{N}^{-x y}|y\rangle
$$

Then we can expand out this sum

$$
|x\rangle \rightarrow \frac{1}{\sqrt{2^{n}}} \sum_{y_{1}, y_{2}, \ldots, y_{n} \in\{0,1\}} \omega_{N}^{-x \sum_{k=1}^{n} 2^{n-k} y_{k}}\left|y_{1}, y_{2}, \ldots, y_{n}\right\rangle
$$

## Expanding the exponential

Expanding the exponential of a sum to a product of exponentials and
collecting these terms in from the appropriate terms we can express
this as

$$
|x\rangle \rightarrow \frac{1}{\sqrt{2^{n}}} \sum_{y_{1}, y_{2}, \ldots, y_{n} \in\{0,1\}} \bigotimes_{k=1}^{n} \omega_{N}^{-x 2^{n-k} y_{k}}\left|y_{k}\right\rangle
$$

## Rearranging

We can rearrange the sum and products as

$$
|x\rangle \rightarrow \frac{1}{\sqrt{2^{n}}} \bigotimes_{k=1}^{n}\left[\sum_{y_{k} \in\{0,1\}} \omega_{N}^{-x 2^{n-k} y_{k}}\left|y_{k}\right\rangle\right]
$$

Expanding this sum yields

$$
|x\rangle \rightarrow \frac{1}{\sqrt{2^{n}}} \bigotimes_{k=1}^{n}\left[|0\rangle+\omega_{N}^{-x 2^{n-k}}|1\rangle\right]
$$

## Notation for binary fraction

But now notice that $\omega_{N}^{-x 2^{n-k}}$ is not dependent on the
higer order bits of $x$. It is convenient to adopt the following
expression for a binary fraction:

$$
0 . x_{l} x_{l+1} \ldots x_{n}=\frac{x_{l}}{2}+\frac{x_{l+1}}{4}+\cdots+\frac{x_{n}}{2^{n-l+1}}
$$

## More notations
Then we can see that

$$
|x\rangle \rightarrow \frac{1}{\sqrt{2^{n}}}\left[|0\rangle+e^{-2 \pi i 0 . x_{n}}|1\rangle\right] \otimes\left[|0\rangle+e^{-2 \pi i 0 . x_{n-1} x_{n}}|1\rangle\right] \otimes \cdots \otimes\left[|0\rangle+e^{-2 \pi i 0 . x_{1} x_{2} \cdots x_{n}}|1\rangle\right]
$$

This is a very useful form of the QFT for $N=2^{n}$. Why? Because we
see that only the last qubit depends on the the values of all the
other input qubits and each further bit depends less and less on the
input qubits. Further we note that $e^{-2 \pi i 0 . a}$ is either +1
or -1 , which reminds us of the Hadamard transform.

## Deriving a circuit

So how do we use this to derive a circuit for the QFT over $N=2^{n}$ ?

Take the first qubit of $\left|x_{1}, \ldots, x_{n}\right\rangle$ and
apply a Hadamard transform. This produces the transform

$$
|x\rangle \rightarrow \frac{1}{\sqrt{2}}\left[|0\rangle+e^{-2 \pi i 0 \cdot x_{1}}|1\rangle\right] \otimes\left|x_{2}, x_{3}, \ldots, x_{n}\right\rangle
$$

## Rotation gate
Now define the rotation gate

$$
R_{k}=\left[\begin{array}{cc}
1 & 0  \tag{30}\\
0 & \exp \left(\frac{-2 \pi i}{2^{k}}\right)
\end{array}\right]
$$

If we now apply controlled $R_{2}, R_{3}$, etc. gates controlled on
the appropriate bits this enacts the transform

$$
|x\rangle \rightarrow \frac{1}{\sqrt{2}}\left[|0\rangle+e^{-2 \pi i 0 . x_{1} x_{2} \ldots x_{n}}|1\rangle\right] \otimes\left|x_{2}, x_{3}, \ldots, x_{n}\right\rangle
$$

## Proceeding

Thus we have reproduced the last term in the QFTed state. Of course
now we can proceed to the second qubit, perform a Hadamard, and the
appropriate controlled $R_{k}$ gates and get the second to last
qubit. Thus when we are finished we will have the transform

$$
|x\rangle \rightarrow \frac{1}{\sqrt{2^{n}}}\left[|0\rangle+e^{-2 \pi i 0 . x_{1} x_{2} \cdots x_{n}}|1\rangle\right] \otimes\left[|0\rangle+e^{-2 \pi i 0 . x_{1} x_{2} \cdots x_{n-1}}|1\rangle\right] \otimes \cdots \otimes\left[|0\rangle+e^{-2 \pi i 0 . x_{n}}|1\rangle\right]
$$

Reversing the order of these qubits will then produce the QFT!

<!-- The circuit we have constructed on $n$ qubits is -->

<!-- \includegraphics[max width=\textwidth]{2024_03_18_c1d427fa6b85efa365f1g-5} -->

## Plans for the week of April 8-12

1. Finalize our discussion of QFTs

2. Set up calculations of QFTs

3. Implementing the phase estimation algorithm for finding eigenvalues

## From DFT to QFT

The Quantum Fourier Transform (QFT) has mathematically the same equation as starting point
but the notation is generally different. We generally compute the
quantum Fourier transform on a set of orthonormal basis state vectors
\(|0\rangle, |1\rangle, ..., |N-1\rangle\). The linear operator defining
the transform is given by the action on basis states

$$
|j\rangle \mapsto \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} e^{2\pi ijk/N}|k\rangle.
$$

## In terms of arbitrary states

This can be written on arbitrary states,

$$
\sum_{j=0}^{N-1}x_j|j\rangle \mapsto \sum_{k=0}^{N-1} y_k|k\rangle
$$

where each amplitude \(y_k\) is the discrete Fourier transform of
\(x_j\).

## Unitarity

The quantum Fourier transform is unitary. Taking \(N = 2^n\),
for \(n\) qubits gives us the orthonormal (computational) basis

$$
|0\rangle, |1\rangle, ..., |2^{n}-1\rangle.
$$

## Binary representation

Each of the computational basis states can be represented in binary

$$
j = j_1j_2 \cdots j_n
$$

where each \(j_k\) is either \(0\) or \(1\), and the corresponding
binary vector is \(|j_1j_2 \cdots j_n\rangle\).

## Rewriting the QFT

The quantum Fourier
transform on one of these \(n\)-qubit vectors can be written as,

$$
\begin{align*}
|j_1j_2 \cdots j_n \rangle = \frac{\left(|0\rangle +e^{2\pi i0.j_n}|1\rangle\right) \otimes \left(|0\rangle +e^{2\pi i0.j_{n-1}j_n}|1\rangle\right) \otimes \cdots \otimes \left(|0\rangle +e^{2\pi i0.j_1j_2 \cdots j_n}|1\rangle\right)}{2^{n/2}}
\end{align*}
$$

## Short hand notation

In the above, we use the notation

$$
0.j_lj_{l+1} \cdots j_n = \frac{j+l}{2} + \frac{j_{l+1}}{2^2} + \cdots + \frac{j_n}{2^{m-l+1}}
$$

## Two-qubit system
First a two qubit system and write code for it as well.

## Four qubit system
Let's have a look at an example on a four qubit system. The basis states are,

$$
|j_1j_2j_3j_4 \rangle
$$

where \(j_k\) is either \(0\) or \(1\). We have,

$$
\begin{align*}
0.j_3 &= \frac{j_3}{2} \\
0.j_2j_3 &= \frac{j_2}{2} + \frac{j_3}{4} \\
0.j_1j_2j_3 &= \frac{j_1}{2} + \frac{j_2}{4} + \frac{j_3}{8} \\
0.j_0j_1j_2j_3 &= \frac{j_0}{2} + \frac{j_1}{4} + \frac{j_2}{8} + \frac{j_3}{16} \\
\end{align*}
$$

## QFT acts as follows

The quantum Fourier transform acts as follows:

$$
\begin{align*}
|j_1j_2j_3j_4 \rangle \mapsto \frac{1}{\sqrt{2^{4/2}}}
\left(|0\rangle + e^{2 \pi i 0.j4}|1\rangle \right) \otimes 
\left(|0\rangle + e^{2 \pi i 0.j_3j4}|1\rangle \right) \otimes 
\left(|0\rangle + e^{2 \pi i 0.j_2j_3j4}|1\rangle \right) \otimes 
\left(|0\rangle + e^{2 \pi i 0.j_1j_2j_3j4}|1\rangle \right) 
\end{align*}
$$

## Quantum circuits

To compose a quantum circuit that calculates the quantum Fourier
transform we use the operators

$$
\begin{align*}
R_k = 
\begin{bmatrix}
1 & 0 \\
0 & e^{2\pi i/2^k}
\end{bmatrix}.
\end{align*}
$$

The circuit computing the QFT on four qubits is given by (add more)

## Rotation gates

In this example, the \(R_k\) gates are:

$$
\begin{align*}
R_1 = \begin{bmatrix}
1 & 0 \\
0 & e^{2 \pi i /2^0}
\end{bmatrix} = 
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}, \quad 
R_2 = \begin{bmatrix}
1 & 0 \\
0 & e^{2 \pi i /2^2}
\end{bmatrix}, \quad 
R_3 = \begin{bmatrix}
1 & 0 \\
0 & e^{2 \pi i /2^3}
\end{bmatrix}, \quad 
R_4 = \begin{bmatrix}
1 & 0 \\
0 & e^{2 \pi i /2^4}
\end{bmatrix}.
\end{align*}
$$